# Imports

In [1]:
import torch
# from torch.utils.data import Datasets
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn

Initializing Training Data and Testing Data

In [2]:
training_data = datasets.FashionMNIST(root = "data", train = True, download = True, transform = ToTensor())
testing_data = datasets.FashionMNIST(root = "data", train = False, download = True, transform = ToTensor())

100%|██████████| 26421880/26421880 [00:02<00:00, 12474384.55it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209457.95it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3882056.72it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7897687.27it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Shape of the dataset

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(testing_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X : {X.shape}")
    print(f"shape of y : {y.dtype}")
    break   

Shape of X : torch.Size([64, 1, 28, 28])
shape of y : torch.int64


Setting the Device

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"using {device} device")

using cpu device


Defining Model

In [5]:
class neuralnetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = neuralnetwork().to(device)
print(model)

neuralnetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Loss Function

In [6]:
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [7]:
def train(dataloader, model, loss_fun, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fun(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss : {loss} [{current}/{size}]")

In [8]:
def test(dataloader, model, loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss +=loss_fun(pred, y).item()
            correct +=(pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy : {(100*correct)}, Average Loss : {test_loss}\n")

In [9]:
epochs = 5

for t in range(epochs):
    print(f"Epochs {t+1}\n")
    train(train_dataloader, model, loss_fun, optimizer)
    test(test_dataloader, model, loss_fun)
    
print("Training and Testing over")

Epochs 1

loss : 2.295952558517456 [64/60000]
loss : 2.289655923843384 [6464/60000]
loss : 2.267094850540161 [12864/60000]
loss : 2.263235092163086 [19264/60000]
loss : 2.24517822265625 [25664/60000]
loss : 2.208731174468994 [32064/60000]
loss : 2.2231953144073486 [38464/60000]
loss : 2.1900484561920166 [44864/60000]
loss : 2.181304693222046 [51264/60000]
loss : 2.1392862796783447 [57664/60000]
Test Error : 
 Accuracy : 38.29, Average Loss : 2.14236981853558

Epochs 2

loss : 2.153245449066162 [64/60000]
loss : 2.1455278396606445 [6464/60000]
loss : 2.083587646484375 [12864/60000]
loss : 2.102672576904297 [19264/60000]
loss : 2.04915714263916 [25664/60000]
loss : 1.986999273300171 [32064/60000]
loss : 2.0129506587982178 [38464/60000]
loss : 1.9360803365707397 [44864/60000]
loss : 1.9329363107681274 [51264/60000]
loss : 1.8569904565811157 [57664/60000]
Test Error : 
 Accuracy : 56.97, Average Loss : 1.86182401818075

Epochs 3

loss : 1.8915159702301025 [64/60000]
loss : 1.86385726928710

In [10]:
torch.save(model.state_dict(), "Model.pth")

print("Model saved as Model.pth")

Model saved as Model.pth


In [11]:
model = neuralnetwork().to(device)
model.load_state_dict(torch.load("Model.pth"))

<All keys matched successfully>

In [12]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

model.eval()

x,y = testing_data[0][0], testing_data [0][1]

with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    
    prediction, actual = classes[pred[0].argmax(0)], classes[y]
    
    print(f"Predicted : {prediction}, Actual : {actual}")

Predicted : Ankle boot, Actual : Ankle boot
